conda install -c conda-forge pandas-profiling

In [1]:

import sys
import pandas as pd 
from pandas_profiling import ProfileReport
#import tensorflow as tf 
#import data_preparer

# df = pd.read_csv('D:/projects/ai/poc/homework/training_data_20210128.csv')
# df_test = pd.read_csv('D:/projects/ai/poc/homework/testing_data_20210128.csv')

# print(df.columns)
# print(test_df.columns)

In [2]:
df_train = pd.read_csv('D:/projects/ai/poc/homework/training_data_20210128.csv')
# df=df.head(1000)
#df = pd.read_excel(source_file_path)
#df = pd.read_csv(source_file_path)
# df_train = df_train.loc[df_train['LOT_TYPE']=='FDY']
df_train = df_train.loc[df_train['LAYER']!='XX']
df_train['CHIPNAME'] = df_train['CHIPNAME'].str.strip()
        

df_train['PRIORITY'] = df_train['PRIORITY'].astype(str)
df_train['IS_MAIN_ROUTE'] = df_train['IS_MAIN_ROUTE'].astype(str)
df_train['DATA_DATE'] = df_train['DATA_DATE'].astype(str)
          
df_train['DATA_DATE'] = pd.to_datetime(df_train['DATA_DATE'], infer_datetime_format=True)
df_train['WS_DATE'] = pd.to_datetime(df_train['WS_DATE'], infer_datetime_format=True)
df_train = df_train.assign(PROCESSED_DAYS = ((df_train['DATA_DATE'] -df_train['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_train['ACTUAL_WP_OUT'] =pd.to_datetime(df_train['ACTUAL_WP_OUT'], infer_datetime_format=True)
df_train = df_train.assign(REMAIN_DAYS=((df_train['ACTUAL_WP_OUT']-df_train['DATA_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_train = df_train.assign(CYCLE_TIME=((df_train['ACTUAL_WP_OUT']-df_train['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))


In [3]:
def iqr(df, colname, bounds = [0, .75]):
    s = df[colname]
    q = s.quantile(bounds)
    return df[~s.clip(*q).isin(q)]
#df_train =iqr(df_train,'CYCLE_TIME')    

In [4]:
df_test = pd.read_csv('D:/projects/ai/poc/homework/testing_data_20210128.csv')
# df_test = df_test.loc[df_test['LOT_TYPE']=='FDY']
df_test = df_test.loc[df_test['LAYER']!='XX']
df_test['CHIPNAME'] = df_test['CHIPNAME'].str.strip()

df_test['PRIORITY'] = df_test['PRIORITY'].astype(str)
df_test['IS_MAIN_ROUTE'] = df_test['IS_MAIN_ROUTE'].astype(str)
df_test['DATA_DATE'] = df_test['DATA_DATE'].astype(str) 

df_test['DATA_DATE'] = pd.to_datetime(df_test['DATA_DATE'], infer_datetime_format=True)

df_test['WS_DATE'] = pd.to_datetime(df_test['WS_DATE'], infer_datetime_format=True)
df_test = df_test.assign(PROCESSED_DAYS = ((df_test['DATA_DATE'] -df_test['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_test['ACTUAL_WP_OUT'] =pd.to_datetime(df_test['ACTUAL_WP_OUT'], infer_datetime_format=True)
df_test = df_test.assign(REMAIN_DAYS=((df_test['ACTUAL_WP_OUT']-df_test['DATA_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_test = df_test.assign(CYCLE_TIME=((df_test['ACTUAL_WP_OUT']-df_test['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))

In [5]:
#df_test =iqr(df_test,'CYCLE_TIME')    

In [6]:
df_train.columns

Index(['DATA_DATE', 'IDX', 'LOT_ID', 'STATUS', 'CHIPNAME', 'LAYER',
       'REMAIN_LAYER_SEQ', 'OP_NO', 'REMAIN_OP_SEQ', 'PRIORITY',
       ...
       'UM', 'UT', 'WL', 'WS', 'WT', 'ZL', 'ACTUAL_WP_OUT', 'PROCESSED_DAYS',
       'REMAIN_DAYS', 'CYCLE_TIME'],
      dtype='object', length=263)

In [7]:
 #df = df.drop(columns=['IDX','LOT_TYPE','WS_DATE','ACTUAL_WP_OUT','DATA_DATE','LAYER','LOT_ID'])
 df_train = df_train.drop(df_train.loc[:, '0E':'ZL'].columns, axis = 1) 

 #df_test = df_test.drop(columns=['IDX','LOT_TYPE','WS_DATE','ACTUAL_WP_OUT','DATA_DATE','LAYER','LOT_ID'])
 df_test = df_test.drop(df_test.loc[:, '0E':'ZL'].columns, axis = 1) 


print(df_test.columns)         

Index(['DATA_DATE', 'IDX', 'LOT_ID', 'STATUS', 'CHIPNAME', 'LAYER',
       'REMAIN_LAYER_SEQ', 'OP_NO', 'REMAIN_OP_SEQ', 'PRIORITY', 'LOT_TYPE',
       'WIP_QTY', 'WS_DATE', 'IS_MAIN_ROUTE', 'ACTUAL_WP_OUT',
       'PROCESSED_DAYS', 'REMAIN_DAYS', 'CYCLE_TIME'],
      dtype='object')


In [8]:
print(df_train.shape)
print(df_train.info())

print(df_test.shape)
print(df_test.info())

(224907, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 224907 entries, 0 to 226722
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DATA_DATE         224907 non-null  datetime64[ns]
 1   IDX               224907 non-null  int64         
 2   LOT_ID            224907 non-null  object        
 3   STATUS            224907 non-null  object        
 4   CHIPNAME          224907 non-null  object        
 5   LAYER             224907 non-null  object        
 6   REMAIN_LAYER_SEQ  224706 non-null  float64       
 7   OP_NO             224907 non-null  object        
 8   REMAIN_OP_SEQ     224636 non-null  float64       
 9   PRIORITY          224907 non-null  object        
 10  LOT_TYPE          224907 non-null  object        
 11  WIP_QTY           224907 non-null  int64         
 12  WS_DATE           224907 non-null  datetime64[ns]
 13  IS_MAIN_ROUTE     224907 non-null  object     

In [9]:
#Categorical Columns like  
#Continuous Columns like  
#Requirement : I need count of output date

# print(pd.crosstab(train_df.out,columns='WIPDT',margins=True))
# print(pd.crosstab(test_df.out,columns='WIPDT',margins=True))

In [10]:
#Requirement: All Numerical columns basic statistics.
df_train.describe()

,IDX,REMAIN_LAYER_SEQ,REMAIN_OP_SEQ,WIP_QTY,PROCESSED_DAYS,REMAIN_DAYS,CYCLE_TIME
count,224907.000000,224706.000000,224636.000000,224907.000000,224907.000000,224907.000000,224907.000000
mean,240346.918469,13.632636,290.624646,22.650006,55.465060,34.262914,89.727974
std,145960.363003,8.787749,174.939457,5.948238,49.549116,20.473583,45.294063
min,73.000000,1.000000,0.000000,1.000000,0.092361,0.004167,6.983333
25%,110996.500000,7.000000,146.000000,25.000000,31.225694,16.772222,70.370833
50%,232569.000000,12.000000,278.000000,25.000000,49.309722,33.696528,82.968750
75%,363074.500000,19.000000,425.000000,25.000000,69.310069,50.536111,97.318750
max,511707.000000,51.000000,1013.000000,25.000000,1993.587500,85.218056,1994.570833


In [11]:
#Requirement: All Numerical columns basic statistics.
df_test.describe()

,IDX,REMAIN_LAYER_SEQ,REMAIN_OP_SEQ,WIP_QTY,PROCESSED_DAYS,REMAIN_DAYS,CYCLE_TIME
count,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000
mean,535860.655721,6.441791,152.621891,23.526368,59.385144,16.809771,76.194915
std,1301.819519,4.036172,104.384624,4.984093,34.295574,11.372813,31.975142
min,531147.000000,1.000000,2.000000,1.000000,8.211806,0.045833,39.134722
25%,534855.000000,3.000000,58.000000,25.000000,42.188194,7.175000,61.643056
50%,535973.000000,7.000000,148.000000,25.000000,53.270139,15.180556,67.936806
75%,536844.000000,9.000000,238.000000,25.000000,69.302083,27.984028,80.071528
max,539293.000000,32.000000,510.000000,25.000000,605.659028,36.232639,629.957639


Pandas Profiling

In [15]:
profile = ProfileReport(df_train, title='MPG Pandas Profiling Report', explorative = True)
#profile.to_html()  #支援輸出html
profile.to_file("output.html")  #支援輸出html

TypeError: to_file() got an unexpected keyword argument 'outputfile'